# Los Angeles Real Estate Price Prediction

## Part 1: Data Cleaning

Real estate prices are something that are tied not only to the attributes of the property, such as the lot size, house size, and other variables, but also the conditions around them that exist in space. 
This could be demographics, nearby sales, neighborhoods, proximity to business districts, etc.

To truly understand and predict prices we need to look at the spatial variables that make up the conditions around a property. In this notebook we will explore that further using real estate sales data from 2017 to 2019 in Los Angeles County.

This data has been imported into CARTO, a spatial data science platform and PostGIS database, and will be brought into the notebook using CARTOFrames, a library that will allow us to interface with data in our CARTO account and create map visualizations. 

The data consists of two datasets, the first being historical sales data from the Los Angeles County Office of the Assessor:    

- [Property Assessment Information System Map](http://maps.assessor.lacounty.gov/GVH_2_2/Index.html?configBase=http://maps.assessor.lacounty.gov/Geocortex/Essentials/REST/sites/PAIS/viewers/PAIS_hv/virtualdirectory/Resources/Config/Default)
- [Sales Parcels](http://assessor.gis.lacounty.gov/assessor/rest/services/PAIS/pais_sales_parcels/MapServer/0)
- [All Parcels](http://assessor.gis.lacounty.gov/assessor/rest/services/PAIS/pais_parcels/MapServer/0)

And parcel information from the LA County Open Data Portal:

- [2018 Property Parcels](https://data.lacounty.gov/Parcel-/Assessor-Parcels-Data-2018/mk7y-hq5p)

Our goals for this project are:


1. To import and clean the real estate data 
2. Exploratory (spatial) data analysis
3. Limit our analysis to one category of property and explore relevant features
4. Feature engineering and tests with various machine learning models
5. Add spatial features to the data and explore spatial relationships
6. Test the model with spatial features to see the impact
7. Evaluate our model and deploy it for production usage


In [ ]:
import pandas as pd
import cartoframes
from cartoframes.contrib import vector
import seaborn as sns
import numpy as np
import libpysal
from cartoframes import Credentials
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.pylabtools import figsize

pd.set_option('display.max_columns', 500)
sns.set_style("white")
sns.set_style("ticks")
sns.despine()


USERNAME = 'mforrest-isolines'  
APIKEY = '07ab3fb439d92c5f06cfec08bb3417d209c646d8'
creds = Credentials(username=USERNAME, key=APIKEY)

cc = cartoframes.CartoContext(creds=creds)

# Import from CARTO

We've joined the sales_parcels with the assersor_parcels_data_2018 on AIN (Assesoor
CREATE TABLE la_join AS * 
    FROM sales_parcels s
    LEFT JOIN assessor_parcels_data_2018 p ON s.ain::numeric = p.ain

In [ ]:
homes = cc.read('la_join')

# Clean our data

As you can see from the data dictionary, we will return many different columns from the joined price data and building footprint boundaries, much of which will not be usefull for our final evaluation. Let's explore, then clean the data we have.

In [ ]:
homes.head()

In [ ]:
homes.describe()

In [ ]:
homes.columns.values

# Clean out missing values

First we need to get rid of null values in the data that will not be usefull for the model later on. Using the `missing_values_table()` function we will identify the columns with the most missing values and then remove them at a specific threshold.

In [ ]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(homes)

In [ ]:
homes.columns.values

# Remove columns

As we stated earlier, many of the columns are not useful for our later analysis. Please refer to the two data dictionaries at the beginning of the notebook to see the specific definitions, but we are removing mostly duplicated and other columns that we don't need later on.

- `ain_p`, `formatted_ain`, and `assessorid`: Same as `ain`
- `bathrooms_p`: Duplicate
- `bedrooms_p`: Duplicate
- `effective_yearbuilt`: Duplicate
- `objectid`: Random identifier
- `usecode`: Duplicate
- `yearbuilt_p`: Duplicate

In [ ]:
columns = ['ain_p', 'assessorid', 'bathrooms_p',
       'bedrooms_p', 'effective_yearbuilt', 'formatted_ain', 
       'objectid', 'usecode', 'usetype',
       'yearbuilt_p']

homes = homes.drop(columns=columns)
homes.head()

In [ ]:
missing_values_table(homes)

In [ ]:
homes = homes.dropna()
homes.head()

# Review the building categories

As we can see many of the building categories in the data are actually cooperative buildings. From here we can begin to categorize and review specific related subsets of these building types.

In [ ]:
homes['generalusetype'].value_counts()

In [ ]:
homes[homes['generalusetype'] == 'Residential']['specificusetype'].value_counts()

# Filter Single Family Residences

Since the majority of the data is in the 'Single Family Residence' category, let's focus our analysis on those buildings.

In [ ]:
sfr = homes[homes['specificusetype'] == 'Single Family Residence']
sfr.head()

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})

sns.heatmap(sfr.corr(),cmap='magma',linecolor='white',linewidths=2)

# Map Single Family Residential

In [ ]:

qSFR = '''
      SELECT * FROM la_join
      WHERE specificusetype = 'Single Family Residence'
      '''

colorRamp='ramp(viewportStandardDev($saleprice, 7), sunset)';
strokeRamp='ramp(viewportStandardDev($saleprice, 7), [#D9CE8B,#E0B177,#DE9071,#D17177,#B55B82,#874C87,#4E468C])';

vector.vmap(
    [vector.QueryLayer(
        qSFR,
        color='ramp(zoomrange([0,16]),[opacity('+colorRamp+',1),opacity('+colorRamp+',0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.8])',
        strokeColor=strokeRamp,
        interactivity={
            'cols': ['formatted_saleprice','formatted_size'],
            'header': ['<h2>Single Family Residence<h2>', ],
            'event': 'hover'
        }
    ),],
    context=cc,
    basemap=vector.BaseMaps.voyager
)

# High Sale Homes

Most of the homes over $5M are in Beverly Hills, Santa Monica, Malibu, and Hollywood.

In [ ]:
q5m = '''
      SELECT * FROM la_join
      WHERE specificusetype = 'Single Family Residence'
      AND saleprice > 10000000
      '''
colorRamp='ramp(viewportStandardDev($saleprice, 7), sunset)';
strokeRamp='ramp(viewportStandardDev($saleprice, 7), [#D9CE8B,#E0B177,#DE9071,#D17177,#B55B82,#874C87,#4E468C])';

vector.vmap(
    [vector.QueryLayer(
        q5m,
        color='ramp(zoomrange([0,16]),[opacity('+colorRamp+',1),opacity('+colorRamp+',0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor=strokeRamp,
        interactivity={
            'cols': ['formatted_saleprice','formatted_size'],
            'header': ['<h2>Single Family Residence > 5 Million<h2>', ],
            'event': 'hover'
        }
    ),
    ],
    context=cc,
    basemap=vector.BaseMaps.darkmatter
)

# Low Sale Homes

Homes under $300K seem to be dispersed throughout the city.

In [ ]:
q300K = '''
      SELECT * FROM la_join
      WHERE specificusetype = 'Single Family Residence'
      AND saleprice < 300000
      '''
colorRamp='ramp(viewportStandardDev($saleprice, 7), sunset)';
strokeRamp='ramp(viewportStandardDev($saleprice, 7), [#D9CE8B,#E0B177,#DE9071,#D17177,#B55B82,#874C87,#4E468C])';


vector.vmap(
    [vector.QueryLayer(
        q300K,
        color='ramp(zoomrange([0,16]),[opacity('+colorRamp+',1),opacity('+colorRamp+',0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor=strokeRamp,
        interactivity={
            'cols': ['formatted_saleprice','formatted_size'],
            'header': ['<h2>Single Family Residence < 300 Thousand<h2>', ],
            'event': 'hover'
        }
    ),
    ],
    context=cc,
    basemap=vector.BaseMaps.voyager
)

# High Value Homes

Many of the high value homes in Los Angeles are tied to celebrity names, such as [320 N Carolwood Dr](https://www.zillow.com/homedetails/320-N-Carolwood-Dr-Los-Angeles-CA-90077/20523956_zpid/) which was reportedly Frank Sinatra's old home and [454 Cuesta Way](https://therealdeal.com/la/2017/04/27/everything-you-need-to-know-about-beyonce-and-jay-zs-prospective-new-home/) which was bid on by Beyonce and Jay-Z.

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sfr[['propertylocation', 'saleprice']].sort_values(by=(['saleprice']), ascending=False).head(10)

# Low Sale Homes

Additionally there are many low value transactions that are outliers we need to account for. It is not clear why they have sold for such low prices, but you can see that the home values are actually much higher or are 'knockdown' lots:

- [10757 Clarkson Rd](https://www.zillow.com/homedetails/10757-Clarkson-Rd-Los-Angeles-CA-90064/20461678_zpid/)
- [14404 S Corlett Ave](https://www.zillow.com/homedetails/14404-S-Corlett-Ave-Compton-CA-90220/20992072_zpid/)
- [15730 Hesby St](https://www.zillow.com/homedetails/15730-Hesby-St-Encino-CA-91436/19980962_zpid/)
- [5131 Gaviota Ave](https://www.zillow.com/homedetails/5131-Gaviota-Ave-Encino-CA-91436/19981445_zpid/)

In [ ]:
sfr[['propertylocation', 'saleprice']].loc[sfr['saleprice'] > 1].sort_values(by=(['saleprice']), ascending=True).head(30)

In [ ]:
sns.set_style("white")
sns.set_style("ticks")

d = pd.Series(np.log(sfr.saleprice))

ax = sns.distplot(d, bins=20, kde=True, rug=True, color="#0A157F", axlabel='Sales Price')

# Write current data to CARTO

To see where these different values are, let's first write this dataframe to CARTO to explore it on the map.

In [ ]:
cc.write(sfr, 'la_singlefamilyhomes', overwrite=True)

In [ ]:
colorRamp='ramp(viewportStandardDev($saleprice, 7), sunset)';
strokeRamp='ramp(viewportStandardDev($saleprice, 7), [#D9CE8B,#E0B177,#DE9071,#D17177,#B55B82,#874C87,#4E468C])';

vector.vmap(
    [vector.Layer(
        'la_singlefamilyhomes',
        color='ramp(zoomrange([0,16]),[opacity('+colorRamp+',1),opacity('+colorRamp+',0.6)])',
        strokeWidth='ramp(zoomrange([12,14]),[0,0.7])',
        strokeColor=strokeRamp,
        interactivity={
            'cols': ['formatted_saleprice','formatted_size'],
            'header': ['<h2>LA Single Family Residence<h2>', ],
            'event': 'hover'
        }
    ),
    ],
    context=cc,
    basemap=vector.BaseMaps.voyager
)